In [1]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)
from glmnet.cox import CoxFamily, CoxLM
import rpy2
%load_ext rpy2.ipython

In [2]:
n, p = 200, 10
event = rng.integers(0, 10, size=n) + rng.standard_exponential(size=n)
status = rng.choice([0, 1], size=n)
event_data = pd.DataFrame({'event':event,
                           'status':status})
X = rng.standard_normal((n, p))

In [3]:
%%R -i event,status,X
library(survival)
Y = Surv(event, status)
M = coxph(Y ~ X)
M


Call:
coxph(formula = Y ~ X)

        coef exp(coef) se(coef)      z     p
X1   0.06344   1.06550  0.10203  0.622 0.534
X2  -0.08998   0.91395  0.10526 -0.855 0.393
X3   0.04800   1.04917  0.09298  0.516 0.606
X4   0.13261   1.14181  0.10410  1.274 0.203
X5  -0.04312   0.95780  0.10803 -0.399 0.690
X6  -0.07398   0.92869  0.11227 -0.659 0.510
X7   0.01283   1.01291  0.09734  0.132 0.895
X8  -0.04905   0.95213  0.12254 -0.400 0.689
X9   0.01888   1.01906  0.08937  0.211 0.833
X10  0.10674   1.11265  0.10712  0.996 0.319

Likelihood ratio test=4.39  on 10 df, p=0.9282
n= 200, number of events= 102 


In [4]:
cox_fam = CoxFamily()

In [5]:
G = CoxLM(family=cox_fam)
G.fit(X, event_data)

got it
[  5.58796256   1.86271109  27.57261624  -2.55199852 -13.64812678
   1.23923511  17.6214929   -9.95519053   1.36650952 -14.97879164] coefnew
[  5.58796256   1.86271109  27.57261624  -2.55199852 -13.64812678
   1.23923511  17.6214929   -9.95519053   1.36650952 -14.97879164]
51.109179414166235 finite
[  5.58796256   1.86271109  27.57261624  -2.55199852 -13.64812678
   1.23923511  17.6214929   -9.95519053   1.36650952 -14.97879164]
51.109179414166235 valid
[  5.58796256   1.86271109  27.57261624  -2.55199852 -13.64812678
   1.23923511  17.6214929   -9.95519053   1.36650952 -14.97879164]
51.109179414166235 2.1998998855173113 decr
51.109179414166235 2.1998998855173113 decr
25.568258665524787 2.1998998855173113 decr
12.8335483997694 2.1998998855173113 decr
6.5372164558864805 2.1998998855173113 decr
3.6103250214210125 2.1998998855173113 decr
2.562195209617542 2.1998998855173113 decr
2.2851785159577513 2.1998998855173113 decr
2.2185262869505205 2.1998998855173113 decr
2.2032376838011283

CoxLM(family=CoxFamily(tie_breaking='efron', event_col='event', status_col='status', start_col=None), fit_intercept=False, control=GLMControl(mxitnr=25, epsnr=1e-06, big=9.9e+35, logging=False), summarize=False)

In [6]:
G.coef_

array([ 0.00546787,  0.00182299,  0.02697291, -0.00249635, -0.01335151,
        0.00121074,  0.01723716, -0.00973723,  0.00133659, -0.01465245])

In [7]:
%%R
source('/Users/jtaylo/git-repos/glmnet/R/coxpath.R')
